# **Installing Dependencies**

In [11]:
!pip install catboost optuna

# **Importing Necessary Libraries and Setup**

In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision.models import resnet18
from torchvision import transforms
import pandas as pd
import numpy as np
import joblib
import warnings
from tqdm import tqdm
from typing import Tuple
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import QuantileTransformer
from sklearn.metrics import roc_auc_score, roc_curve, precision_score, recall_score, f1_score
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier, early_stopping, log_evaluation
from catboost import CatBoostClassifier
import optuna
optuna.logging.set_verbosity(optuna.logging.WARNING)

# Warnings
warnings.filterwarnings('ignore')

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


# **Defining Image Transform and Feature Extractor**

In [13]:
# Normalization stats for the dataset
mean = [0.2980, 0.2962, 0.2987]
std = [0.2886, 0.2875, 0.2889]

class CustomTransform:
    """
    Applies basic data augmentation, normalization, and moves image to the correct device.
    """
    def __init__(self, mean, std):
        self.to_tensor = transforms.ToTensor()
        self.normalize = transforms.Normalize(mean=mean, std=std)
        self.augment = transforms.Compose([
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.ColorJitter(brightness=0.1, contrast=0.1)
        ])

    def __call__(self, img):
        if isinstance(img, torch.Tensor):
            img_tensor = img
        else:
            img_tensor = self.to_tensor(self.augment(img))

        if img_tensor.dim() == 3 and img_tensor.shape[0] not in [1, 3]:
            img_tensor = img_tensor.permute(2, 0, 1)

        return self.normalize(img_tensor).to(device)

transform = CustomTransform(mean=mean, std=std)

class FeatureExtractor(nn.Module):
    """
    Extracts both conv5 features and final logits from ResNet18.
    Used for creating input features for the attack model.
    """
    def __init__(self, model):
        super().__init__()
        self.features = nn.Sequential(*list(model.children())[:-2])
        self.pool = model.avgpool
        self.fc = model.fc

    def forward(self, x):
        conv_features = self.pool(self.features(x)).flatten(1)
        logits = self.fc(conv_features)
        return conv_features, logits

# **Load Model and Define Datasets**

In [14]:
# Load victim model (ResNet18) with custom classification head and pretrained weights
model = resnet18(weights=None)
model.fc = nn.Linear(512, 44)
ckpt = torch.load("./01_MIA.pt", map_location=device)
model.load_state_dict(ckpt)

# Wrap the model to extract both conv5 features and logits
extractor = FeatureExtractor(model).to(device)
extractor.eval()

# Dataset class to store IDs, images, and labels with optional transforms
class TaskDataset(Dataset):
    def __init__(self, transform=None):
        self.ids = []
        self.imgs = []
        self.labels = []
        self.transform = transform

    def __getitem__(self, index) -> Tuple[int, torch.Tensor, int]:
        id_ = self.ids[index]
        img = self.imgs[index]
        if self.transform:
            img = self.transform(img)
        label = self.labels[index]
        return id_, img, label

    def __len__(self):
        return len(self.ids)

# Extended dataset to include membership labels for MIA
class MembershipDataset(TaskDataset):
    def __init__(self, transform=None, is_private=False):
        super().__init__(transform)
        self.membership = []
        self.is_private = is_private

    def __getitem__(self, index) -> Tuple[int, torch.Tensor, int, int]:
        id_, img, label = super().__getitem__(index)
        if self.is_private:
            return id_, img, label
        return id_, img, label, self.membership[index]

# Enable safe loading of MembershipDataset from torch.load
torch.serialization.add_safe_globals({"MembershipDataset": MembershipDataset})

# Load public (training) and private (test) datasets and assign appropriate flags and transforms
public_data = torch.load("./pub.pt", weights_only=False)
priv_data = torch.load("./priv_out.pt", weights_only=False)
public_data.is_private = False
priv_data.is_private = True
public_data.transform = transform
priv_data.transform = transform

# **Feature Extraction and Engineering**

In [15]:
# Enhanced feature extraction from model outputs for MIA training
def extract_features(extractor, imgs, labels):
    """
    Extracts a comprehensive set of features from the victim model, including:
    - Logits and softmax-based statistics (e.g. confidence, margin, entropy)
    - Gradient-based metrics (e.g. norm, std)
    - Internal conv5 representations
    These are used to train a membership inference model.
    """
    extractor.eval()
    imgs = imgs.to(device)
    labels = labels.to(device)

    with torch.no_grad():
        conv_features, outputs = extractor(imgs)
        probs = F.softmax(outputs, dim=1)

        # Standard confidence/entropy-based features
        confidences = torch.max(probs, dim=1)[0]
        true_prob = probs[torch.arange(len(labels)), labels]
        losses = F.cross_entropy(outputs, labels, reduction='none')
        entropies = -torch.sum(probs * torch.log(probs + 1e-8), dim=1)

        # Top-k softmax statistics
        topk_probs, topk_indices = torch.topk(probs, k=5, dim=1)
        margins = topk_probs[:, 0] - topk_probs[:, 1]
        prob_diff = topk_probs[:, 0] - topk_probs[:, 2]
        prob_ratio = topk_probs[:, 0] / (topk_probs[:, 1] + 1e-8)

        # Logit-based features
        true_logits = outputs[torch.arange(len(labels)), labels]
        logit_norm = torch.norm(outputs, p=2, dim=1)
        logit_std = torch.std(outputs, dim=1)
        max_logit = torch.max(outputs, dim=1)[0]
        diff_max_true_logit = max_logit - true_logits
        logit_percentiles = torch.quantile(outputs, torch.tensor([0.25, 0.5, 0.75], device=device), dim=1)

        # Softmax shape indicators
        prob_skew = (probs - probs.mean(dim=1, keepdim=True)).pow(3).mean(dim=1) / (probs.std(dim=1).pow(3) + 1e-8)
        prob_kurtosis = (probs - probs.mean(dim=1, keepdim=True)).pow(4).mean(dim=1) / (probs.std(dim=1).pow(4) + 1e-8) - 3
        prob_max_min = topk_probs[:, 0] - topk_probs[:, -1]

        # Top-k presence and relative comparison features
        true_in_top3 = torch.any(topk_indices == labels.unsqueeze(1), dim=1).float()
        true_in_top5 = torch.any(topk_indices == labels.unsqueeze(1), dim=1).float()
        confidence_minus_true = confidences - true_prob
        true_prob_confidence_ratio = true_prob / (confidences + 1e-8)

    # Gradient-based features (backprop once per batch)
    imgs.requires_grad_(True)
    _, outputs = extractor(imgs)
    loss = F.cross_entropy(outputs, labels)
    loss.backward()
    gradients = imgs.grad
    gradient_norm = torch.norm(gradients, p=2, dim=(1, 2, 3))
    gradient_mean = gradients.abs().mean(dim=(1, 2, 3))
    gradient_std = gradients.std(dim=(1, 2, 3))
    imgs.requires_grad_(False)

    # Prediction correctness and probability variability
    correct = (torch.argmax(outputs, dim=1) == labels).float()
    prob_std = torch.std(probs, dim=1)

    # Return all features in a dictionary
    features = {
        'loss': losses.cpu(),
        'margin': margins.cpu(),
        'true_logit': true_logits.cpu(),
        'gradient_norm': gradient_norm.cpu(),
        'gradient_mean': gradient_mean.cpu(),
        'gradient_std': gradient_std.cpu(),
        'confidence': confidences.cpu(),
        'entropy': entropies.cpu(),
        'correct': correct.cpu(),
        'prob_std': prob_std.cpu(),
        'top1_prob': topk_probs[:, 0].cpu(),
        'top2_prob': topk_probs[:, 1].cpu(),
        'top3_prob': topk_probs[:, 2].cpu(),
        'top5_prob': topk_probs[:, 4].cpu(),
        'true_prob': true_prob.cpu(),
        'logit_norm': logit_norm.cpu(),
        'logit_std': logit_std.cpu(),
        'logit_p25': logit_percentiles[0].cpu(),
        'logit_p50': logit_percentiles[1].cpu(),
        'logit_p75': logit_percentiles[2].cpu(),
        'diff_max_true_logit': diff_max_true_logit.cpu(),
        'true_in_top3': true_in_top3.cpu(),
        'true_in_top5': true_in_top5.cpu(),
        'confidence_minus_true': confidence_minus_true.cpu(),
        'true_prob_confidence_ratio': true_prob_confidence_ratio.cpu(),
        'prob_diff': prob_diff.cpu(),
        'prob_ratio': prob_ratio.cpu(),
        'prob_skew': prob_skew.cpu(),
        'prob_kurtosis': prob_kurtosis.cpu(),
        'prob_max_min': prob_max_min.cpu(),
        'conv_features': conv_features.cpu()
    }
    return features


def add_derived_features(row, conv_features):
    """
    Adds additional hand-engineered features and concatenates them with conv5 features.
    These combine existing stats in ways that help the model identify subtle patterns.
    """
    row = [row[k].item() for k in range(len(row))]
    new_features = [
        np.log(row[0] + 1e-5),
        row[6] / (row[7] + 1e-5),
        row[1] / (row[2] + 1e-5),
        row[14] * row[6],
        (row[3] + 1e-5) / (row[16] + 1e-5),
        row[20] * row[8],
        row[6] - row[14],
        row[14] / (row[10] + 1e-5),
        row[10] - row[11],
        row[11] - row[12],
        row[12] - row[13],
        row[16] / (row[15] + 1e-5),
        row[4] / (row[5] + 1e-5),
        row[27] * row[28],
        row[17] - row[19],
        row[6] / (row[7] + 1e-5),
        row[14] / (row[10] - row[14] + 1e-5),
        row[3] * row[16],
        abs(row[10] - row[14]),
        (row[10] + row[11]) / (row[14] + 1e-5),
        row[3] / (row[4] + 1e-5)
    ]
    return np.concatenate([row, new_features, conv_features])

# **Feature Extraction from Public Data**

In [16]:
# Extract feature vectors and membership labels from the public dataset
public_loader = DataLoader(public_data, batch_size=64, shuffle=False)
X_train, y_train = [], []

for ids, imgs, labels, memberships in tqdm(public_loader, desc="Processing public data"):
    imgs = imgs.to(device)
    labels = labels.to(device)
    features = extract_features(extractor, imgs, labels)

    # Process each sample in the batch
    for j in range(len(ids)):
        # Combine selected features and conv5 features
        row = [features[k][j] for k in features if k != 'conv_features']
        row = add_derived_features(row, features['conv_features'][j].numpy())
        X_train.append(row)
        y_train.append(memberships[j].item())

# Convert to NumPy arrays for model training
X_train = np.array(X_train)
y_train = np.array(y_train)

Processing public data: 100%|██████████| 313/313 [00:19<00:00, 16.39it/s]


# **Feature Selection and Preprocessing**

In [17]:
# Select top 50 most informative features using ANOVA F-score
selector = SelectKBest(f_classif, k=50)
X_train_selected = selector.fit_transform(X_train, y_train)
selected_features = selector.get_support(indices=True)

# Check class balance in the training labels
print("Class distribution in y_train:")
print(pd.Series(y_train).value_counts(normalize=True))

# Normalize selected features using quantile transformation to Gaussian distribution
scaler = QuantileTransformer(output_distribution='normal', random_state=42)
X_scaled = scaler.fit_transform(X_train_selected)

Class distribution in y_train:
0    0.5
1    0.5
Name: proportion, dtype: float64


# **Evaluation Functions**

In [18]:
def calculate_metrics(y_true, y_pred, y_score):
    auc = roc_auc_score(y_true, y_score)
    fpr, tpr, thresholds = roc_curve(y_true, y_score)

    results = {
        'AUC': auc,
        'TPR@FPR=0.05': tpr[np.argmin(np.abs(fpr - 0.05))],
        'TPR@FPR=0.01': tpr[np.argmin(np.abs(fpr - 0.01))],
        'Best Threshold': thresholds[np.argmax(tpr - fpr)],
        'Accuracy': (y_pred == y_true).mean(),
        'Precision': precision_score(y_true, y_pred),
        'Recall': recall_score(y_true, y_pred),
        'F1': f1_score(y_true, y_pred)
    }
    return results

# Find decision threshold that gives the closest FPR to a target value (e.g., 0.05)
def optimize_threshold(y_true, y_score, target_fpr=0.05):
    fpr, tpr, thresholds = roc_curve(y_true, y_score)
    idx = np.argmin(np.abs(fpr - target_fpr))
    return thresholds[idx]

# **Optuna Tuner for CatBoost**

In [23]:
def optimize_catboost(X_train, y_train, X_val, y_val):
    """
    Tune CatBoost hyperparameters using Optuna to maximize AUC on validation data.
    """
    def objective(trial):
        # Define search space for hyperparameters
        params = {
            'iterations': trial.suggest_int('iterations', 500, 2000),  # Fewer iterations for speed
            'depth': trial.suggest_int('depth', 4, 8),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.05, log=True),
            'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1, 5),
            'random_strength': trial.suggest_float('random_strength', 0.1, 0.5),
            'task_type': 'GPU' if torch.cuda.is_available() else 'CPU',
            'random_state': 42,
            'verbose': 0,  # Keep training quiet
            'eval_metric': 'AUC'
        }

        # Train and evaluate on validation set
        model = CatBoostClassifier(**params)
        model.fit(X_train, y_train, eval_set=(X_val, y_val), early_stopping_rounds=50, verbose=0)
        best_score = model.get_best_score()
        return best_score['validation']['AUC']

    # Create validation split for optimization
    X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size=0.2, stratify=y_train, random_state=42)

    # Suppress verbose output from Optuna
    optuna.logging.set_verbosity(optuna.logging.WARNING)

    # Optimize hyperparameters
    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=5)  # Adjust n_trials for more thorough tuning

    return study.best_params

# **Ensemble Class**

In [24]:
class Ensemble:
    """Stacking ensemble using XGBoost, LightGBM, and CatBoost as base models,
    and logistic regression as the meta-model."""

    def __init__(self, catboost_params=None):
        # Handle class imbalance by scaling the positive class weight
        scale_pos_weight = (len(y_train) - sum(y_train)) / sum(y_train)

        # Define base models with custom hyperparameters
        self.base_models = [
            XGBClassifier(
                n_estimators=3000,
                max_depth=7,
                learning_rate=0.015,
                gamma=0.2,
                subsample=0.6,
                colsample_bytree=0.6,
                reg_alpha=0.2,
                reg_lambda=1.5,
                scale_pos_weight=scale_pos_weight,
                tree_method='gpu_hist' if torch.cuda.is_available() else 'hist',
                eval_metric='auc',
                early_stopping_rounds=100,
                random_state=42
            ),
            LGBMClassifier(
                n_estimators=3000,
                max_depth=6,
                learning_rate=0.025,
                subsample=0.7,
                colsample_bytree=0.7,
                reg_alpha=0.2,
                reg_lambda=0.2,
                scale_pos_weight=scale_pos_weight,
                metric='auc',
                device='gpu' if torch.cuda.is_available() else 'cpu',
                early_stopping_round=100,
                random_state=42,
                verbosity=-1
            ),
            # CatBoost uses Optuna-tuned or default hyperparameters
            CatBoostClassifier(
                **(catboost_params if catboost_params else {
                    'iterations': 3000,
                    'depth': 7,
                    'learning_rate': 0.025,
                    'l2_leaf_reg': 3,
                    'random_strength': 0.2,
                    'scale_pos_weight': scale_pos_weight,
                    'grow_policy': 'Lossguide',
                    'early_stopping_rounds': 100,
                    'task_type': 'GPU' if torch.cuda.is_available() else 'CPU',
                    'random_state': 42,
                    'verbose': 0,
                    'logging_level': 'Silent'
                })
            )
        ]

        # Meta-model to combine base model outputs
        self.meta_model = LogisticRegression(
            penalty='elasticnet',
            solver='saga',
            l1_ratio=0.5,
            C=0.1,
            max_iter=1000,
            random_state=42
        )

        self.optimal_threshold = None  # Will be set after threshold optimization

    def fit(self, X, y):
        """Train base models using out-of-fold predictions and fit meta-model"""
        kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
        meta_features = np.zeros((len(X), len(self.base_models)))  # Store OOF predictions

        for i, model in enumerate(self.base_models):
            print(f"\nTraining base model {i+1}/{len(self.base_models)} ({model.__class__.__name__})...")
            for train_idx, val_idx in kf.split(X, y):
                X_tr, X_val = X[train_idx], X[val_idx]
                y_tr, y_val = y[train_idx], y[val_idx]

                # Fit each model and get validation predictions
                if isinstance(model, CatBoostClassifier):
                    model.fit(X_tr, y_tr, eval_set=(X_val, y_val), use_best_model=True, verbose=0)
                elif isinstance(model, LGBMClassifier):
                    model.fit(X_tr, y_tr, eval_set=[(X_val, y_val)], callbacks=[early_stopping(100), log_evaluation(100)])
                else:
                    model.fit(X_tr, y_tr, eval_set=[(X_val, y_val)], verbose=False)

                # Store probability predictions for validation fold
                meta_features[val_idx, i] = model.predict_proba(X_val)[:, 1]

        # Train meta-model on stacked predictions
        print("\nTraining meta-model...")
        self.meta_model.fit(meta_features, y)

        # Optimize decision threshold based on TPR@FPR metric
        y_meta = self.meta_model.predict_proba(meta_features)[:, 1]
        self.optimal_threshold = optimize_threshold(y, y_meta)
        print(f"\nOptimal threshold: {self.optimal_threshold:.4f}")

    def predict_proba(self, X):
        """Get meta-model probability predictions from base model outputs"""
        meta_features = np.column_stack([
            model.predict_proba(X)[:, 1] for model in self.base_models
        ])
        return self.meta_model.predict_proba(meta_features)[:, 1]

    def predict(self, X, threshold=None):
        """Make final predictions using optimized or custom threshold"""
        if threshold is None:
            threshold = self.optimal_threshold if self.optimal_threshold else 0.5
        return (self.predict_proba(X) >= threshold).astype(int)

# **Training Ensemble Model**

In [25]:
# Tune CatBoost hyperparameters using Optuna (on same data due to resource limits)
print("\nTuning CatBoost hyperparameters")
catboost_params = optimize_catboost(X_scaled, y_train, X_scaled, y_train)

# Create and train the stacking ensemble using tuned CatBoost and default XGBoost/LightGBM
ensemble = Ensemble(catboost_params=catboost_params)
ensemble.fit(X_scaled, y_train)


Tuning CatBoost hyperparameters


Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU



Training base model 1/3 (XGBClassifier)...

Training base model 2/3 (LGBMClassifier)...
Training until validation scores don't improve for 100 rounds
[100]	valid_0's auc: 0.656048
[200]	valid_0's auc: 0.65686
[300]	valid_0's auc: 0.657415
[400]	valid_0's auc: 0.657726
Early stopping, best iteration is:
[360]	valid_0's auc: 0.659085
Training until validation scores don't improve for 100 rounds
[100]	valid_0's auc: 0.654498
[200]	valid_0's auc: 0.653854
Training until validation scores don't improve for 100 rounds
[100]	valid_0's auc: 0.656769
Training until validation scores don't improve for 100 rounds
[100]	valid_0's auc: 0.653314
[200]	valid_0's auc: 0.650569
Early stopping, best iteration is:
[121]	valid_0's auc: 0.654107
Training until validation scores don't improve for 100 rounds
[100]	valid_0's auc: 0.650133

Training base model 3/3 (CatBoostClassifier)...

Training meta-model...

Optimal threshold: 0.6534


# **Evaluation**

In [26]:
# Evaluation

y_train_score = ensemble.predict_proba(X_scaled)
y_train_pred = ensemble.predict(X_scaled)
train_metrics = calculate_metrics(y_train, y_train_pred, y_train_score)

print("\nFinal Training Metrics:")
for metric, value in train_metrics.items():
    print(f"{metric}: {value:.4f}")


Final Training Metrics:
AUC: 0.7269
TPR@FPR=0.05: 0.1662
TPR@FPR=0.01: 0.0502
Best Threshold: 0.5271
Accuracy: 0.5400
Precision: 0.7869
Recall: 0.1097
F1: 0.1926


# **Process Private Test Data for Inference**

In [27]:
# Extract features from the private (test) set
priv_loader = DataLoader(priv_data, batch_size=64, shuffle=False)
X_test, ids_test = [], []

for ids, imgs, labels in tqdm(priv_loader, desc="Processing private data"):
    imgs = imgs.to(device)
    labels = labels.to(device)
    features = extract_features(extractor, imgs, labels)

    for j in range(len(ids)):
        # Prepare input row without conv_features, then add derived + conv features
        row = [features[k][j] for k in features if k != 'conv_features']
        row = add_derived_features(row, features['conv_features'][j].numpy())
        X_test.append(row)
        ids_test.append(ids[j].item())

# Convert to NumPy array and apply the same feature selection and normalization as training
X_test = np.array(X_test)
X_test_selected = selector.transform(X_test)
X_test_scaled = scaler.transform(X_test_selected)

Processing private data: 100%|██████████| 313/313 [00:17<00:00, 17.75it/s]


In [28]:
# Generating Submission

membership_scores = ensemble.predict_proba(X_test_scaled)
submission_df = pd.DataFrame({
    "ids": ids_test,
    "score": np.clip(membership_scores, 0.001, 0.999)
})
submission_df.to_csv("submission.csv", index=False)

In [29]:
# Saving the trained ensemble model and preprocessing tools
joblib.dump(ensemble, "mia_ensemble.pkl")   # Stacked ensemble model
joblib.dump(scaler, "scaler.pkl")           # QuantileTransformer used for scaling
joblib.dump(selector, "selector.pkl")       # Feature selector (top 50 features)

['selector.pkl']